In [1]:
pip install nltk scikit-learn streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
pairs = [
    {
        "tag": "greeting",
        "patterns": [
            "hi", "hello", "hey", "good morning", "good afternoon", "good evening"
        ],
        "responses": [
            "Hello! Welcome to the Udaipur Travel Guide. How can I assist you today?",
            "Hi there! How can I help you with your Udaipur travel plans?"
        ]
    },
    {
        "tag": "tourist_places",
        "patterns": [
            "what are the best tourist places in Udaipur",
            "top attractions in Udaipur",
            "places to visit in Udaipur",
            "what can I see in Udaipur",
            "famous spots in Udaipur"
        ],
        "responses": [
            "Udaipur is famous for its beautiful lakes and palaces. Some must-visit places are:\n"
            "1. City Palace\n"
            "2. Lake Pichola\n"
            "3. Jag Mandir\n"
            "4. Saheliyon Ki Bari\n"
            "5. Bagore Ki Haveli\n"
            "6. Monsoon Palace\n"
            "7. Fateh Sagar Lake\n"
            "Let me know if you want details about any specific place!"
        ]
    },
    {
        "tag": "city_palace",
        "patterns": [
            "tell me about City Palace",
            "what is City Palace",
            "information about City Palace",
            "details about City Palace"
        ],
        "responses": [
            "City Palace is a majestic palace complex in Udaipur. It features:\n"
            "1. Museums showcasing royal artifacts.\n"
            "2. Beautiful courtyards and gardens.\n"
            "3. Stunning views of Lake Pichola.\n"
            "It's a must-visit for history and architecture lovers!"
        ]
    },
    {
        "tag": "budget",
        "patterns": [
            "what is the budget for Udaipur",
            "how much does it cost to visit Udaipur",
            "prices in Udaipur",
            "cost of traveling to Udaipur",
            "Udaipur travel expenses"
        ],
        "responses": [
            "The budget for Udaipur depends on your travel style:\n"
            "1. **Budget Travel**: ₹1000-₹2000 per day (budget hotels, local food, public transport).\n"
            "2. **Mid-Range Travel**: ₹3000-₹5000 per day (3-star hotels, restaurants, private taxis).\n"
            "3. **Luxury Travel**: ₹7000+ per day (5-star hotels, fine dining, private tours).\n"
            "Let me know if you need help with specific expenses!"
        ]
    },
    {
        "tag": "boat_ride_cost",
        "patterns": [
            "how much does a boat ride cost in Lake Pichola",
            "boat ride price",
            "cost of boat ride in Udaipur",
            "price for boat ride in Lake Pichola"
        ],
        "responses": [
            "The boat ride in Lake Pichola costs around ₹400-₹600 per person for a basic ride. Private boat rides can cost more."
        ]
    },
    {
        "tag": "restaurants",
        "patterns": [
            "best restaurants in Udaipur",
            "where to eat in Udaipur",
            "top restaurants in Udaipur",
            "good places to eat in Udaipur",
            "famous restaurants in Udaipur"
        ],
        "responses": [
            "Udaipur has some amazing restaurants:\n"
            "1. **Ambrai**: Offers lakeside dining with a view of City Palace.\n"
            "2. **Upre**: Rooftop restaurant with a view of Lake Pichola.\n"
            "3. **Charcoal By Carlsson**: Known for its continental cuisine.\n"
            "4. **Millets of Mewar**: Great for healthy and organic food.\n"
            "5. **Jheel's Ginger Coffee Bar**: Perfect for coffee lovers.\n"
            "Let me know if you want more recommendations!"
        ]
    },
    {
        "tag": "weather",
        "patterns": [
            "what is the weather like in Udaipur",
            "best time to visit Udaipur",
            "Udaipur weather",
            "climate in Udaipur",
            "when to visit Udaipur"
        ],
        "responses": [
            "Udaipur has a semi-arid climate. Here's the best time to visit:\n"
            "1. **October to March**: Pleasant weather, ideal for sightseeing (15°C to 25°C).\n"
            "2. **April to June**: Hot and dry (up to 40°C).\n"
            "3. **July to September**: Monsoon season with moderate rainfall.\n"
            "Plan your trip accordingly!"
        ]
    },
    {
        "tag": "hotels",
        "patterns": [
            "best hotels in Udaipur",
            "where to stay in Udaipur",
            "hotels in Udaipur",
            "top hotels in Udaipur",
            "luxury hotels in Udaipur"
        ],
        "responses": [
            "Here are some great hotels in Udaipur:\n"
            "1. **The Oberoi Udaivilas**: A luxury hotel with stunning architecture.\n"
            "2. **Taj Lake Palace**: An iconic hotel located in the middle of Lake Pichola.\n"
            "3. **Trident Udaipur**: A mid-range option with beautiful lake views.\n"
            "4. **Jagat Niwas Palace Hotel**: A heritage hotel with a royal feel.\n"
            "5. **Treebo Trend Krishna Niwas**: A budget-friendly option with good amenities.\n"
            "Let me know if you need help with bookings!"
        ]
    },
    {
        "tag": "souvenirs",
        "patterns": [
            "best souvenirs to buy in Udaipur",
            "what to buy in Udaipur",
            "gifts from Udaipur",
            "shopping in Udaipur",
            "traditional items to buy in Udaipur"
        ],
        "responses": [
            "Udaipur is known for its unique souvenirs and gifts:\n"
            "1. **Miniature Paintings**: Handmade paintings depicting Rajasthani culture.\n"
            "2. **Silver Jewelry**: Exquisite designs crafted by local artisans.\n"
            "3. **Pichwai Paintings**: Traditional cloth paintings of Lord Krishna.\n"
            "4. **Rajasthani Handicrafts**: Pottery, textiles, and wooden items.\n"
            "5. **Bandhani Sarees**: Traditional tie-and-dye sarees.\n"
            "You can find these at Hathi Pol Bazaar and Bada Bazaar."
        ]
    },
    {
        "tag": "temples",
        "patterns": [
            "famous temples in Udaipur",
            "temples to visit in Udaipur",
            "religious places in Udaipur",
            "top temples in Udaipur",
            "spiritual places in Udaipur"
        ],
        "responses": [
            "Udaipur has several beautiful temples:\n"
            "1. **Jagdish Temple**: A 17th-century temple dedicated to Lord Vishnu.\n"
            "2. **Eklingji Temple**: A historic temple complex dedicated to Lord Shiva.\n"
            "3. **Neemach Mata Temple**: Located on a hilltop with panoramic views.\n"
            "4. **Sas Bahu Temple**: Known for its intricate carvings.\n"
            "5. **Shri Manshapurna Karni Mata Temple**: A serene temple near the ropeway.\n"
            "Let me know if you need more details!"
        ]
    },
    {
        "tag": "transport",
        "patterns": [
            "how to travel in Udaipur",
            "transport in Udaipur",
            "local transport options in Udaipur",
            "getting around Udaipur",
            "travel options in Udaipur"
        ],
        "responses": [
            "Here are some ways to get around Udaipur:\n"
            "1. **Auto-rickshaws**: Affordable and widely available.\n"
            "2. **Taxis**: Private taxis can be hired for full-day tours.\n"
            "3. **Buses**: Local buses connect major areas.\n"
            "4. **Bike Rentals**: Scooters and bikes can be rented for self-exploration.\n"
            "5. **Boat Rides**: Available on Lake Pichola and Fateh Sagar Lake.\n"
            "Let me know if you need help with transportation!"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": [
            "bye", "goodbye", "see you later", "take care", "quit"
        ],
        "responses": [
            "Thank you for using the Udaipur Travel Guide. Have a great trip!",
            "Goodbye! Safe travels!"
        ]
    }
]

In [5]:
#Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []

for pair in pairs:
    tag = pair['tag']
    for pattern in pair['patterns']:
        tags.append(tag)
        patterns.append(pattern)

# Print the results
print("Tags:", tags)
print("Patterns:", patterns)

Tags: ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'tourist_places', 'tourist_places', 'tourist_places', 'tourist_places', 'tourist_places', 'city_palace', 'city_palace', 'city_palace', 'city_palace', 'budget', 'budget', 'budget', 'budget', 'budget', 'boat_ride_cost', 'boat_ride_cost', 'boat_ride_cost', 'boat_ride_cost', 'restaurants', 'restaurants', 'restaurants', 'restaurants', 'restaurants', 'weather', 'weather', 'weather', 'weather', 'weather', 'hotels', 'hotels', 'hotels', 'hotels', 'hotels', 'souvenirs', 'souvenirs', 'souvenirs', 'souvenirs', 'souvenirs', 'temples', 'temples', 'temples', 'temples', 'temples', 'transport', 'transport', 'transport', 'transport', 'transport', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']
Patterns: ['hi', 'hello', 'hey', 'good morning', 'good afternoon', 'good evening', 'what are the best tourist places in Udaipur', 'top attractions in Udaipur', 'places to visit in Udaipur', 'what can I see in Udaipur', 'famous sp

In [6]:
#Training the model
x=vectorizer.fit_transform(patterns)
y=tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [7]:
#Python function to chat with chatbot
def chatbot(input_text):
  input_text = vectorizer.transform([input_text])
  tag = clf.predict(input_text)[0]
  for intent in pairs:
    if intent['tag'] == tag:
      response = random.choice(intent['responses'])
      return response

In [8]:
#Checking our chatbot
user_input = "What is weather in udaipur in march?"
response = chatbot(user_input)
print(response)

Udaipur has a semi-arid climate. Here's the best time to visit:
1. **October to March**: Pleasant weather, ideal for sightseeing (15°C to 25°C).
2. **April to June**: Hot and dry (up to 40°C).
3. **July to September**: Monsoon season with moderate rainfall.
Plan your trip accordingly!


In [9]:
#Checking our chatbot
user_input = "Which gift I shall buy from udaipur?"
response = chatbot(user_input)
print(response)

Udaipur is known for its unique souvenirs and gifts:
1. **Miniature Paintings**: Handmade paintings depicting Rajasthani culture.
2. **Silver Jewelry**: Exquisite designs crafted by local artisans.
3. **Pichwai Paintings**: Traditional cloth paintings of Lord Krishna.
4. **Rajasthani Handicrafts**: Pottery, textiles, and wooden items.
5. **Bandhani Sarees**: Traditional tie-and-dye sarees.
You can find these at Hathi Pol Bazaar and Bada Bazaar.


In [10]:
#Checking our chatbot
user_input = "Which transport shoul I use?"
response = chatbot(user_input)
print(response)

Here are some ways to get around Udaipur:
1. **Auto-rickshaws**: Affordable and widely available.
2. **Taxis**: Private taxis can be hired for full-day tours.
3. **Buses**: Local buses connect major areas.
4. **Bike Rentals**: Scooters and bikes can be rented for self-exploration.
5. **Boat Rides**: Available on Lake Pichola and Fateh Sagar Lake.
Let me know if you need help with transportation!


In [11]:
user_input = "Bye"
response = chatbot(user_input)
print(response)

Goodbye! Safe travels!


In [12]:
def chatbot(input_text):
    if not input_text.strip():  # Check if input is empty or just whitespace
        return "Please type something so I can help you!"

    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in pairs:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response
    return "I'm sorry, I didn't understand that. Can you please rephrase?"